In [ ]:
!pip -q install langchain openai==0.27.0 tiktoken

# Summarization

History  
Challenges  
Fine-tuning  
Instruct Tuning

In [ ]:
import os

#os.environ["OPENAI_API_BASE"] = "https://proxy.dta.totvs.ai/"
os.environ["OPENAI_API_BASE"] = "https://proxy.dta.totvs.ai/"
os.environ["OPENAI_API_KEY"] = "sk-axyZ_tPhqNPbbywhdhhhKQ"

In [ ]:
!pip show langchain

### Setting up Summarization Chain

In [ ]:
from langchain import OpenAI, PromptTemplate, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate

#llm = OpenAI(temperature=0)
llm = OpenAI(
            openai_api_base="https://proxy.dta.totvs.ai/",
            openai_api_key="sk-axyZ_tPhqNPbbywhdhhhKQ",
            temperature=0,
            model="gpt-4-0125-preview",
        )

print(llm)

In [ ]:
text_splitter = CharacterTextSplitter()

In [ ]:
# load the doc
with open('sentence.txt') as f:
    how_to_win_friends = f.read()
texts = text_splitter.split_text(how_to_win_friends)


In [ ]:
len(texts)

In [ ]:
from langchain.docstore.document import Document

docs = [Document(page_content=t) for t in texts[:4]]

In [ ]:
docs

##  3 types of CombineDocuments Chains

[Taken from the LangChain Docs](https://langchain.readthedocs.io/en/latest/modules/indexes/combine_docs.html)

## Summarize Simple with map_reduce

### Map Reduce
This method involves **an initial prompt on each chunk of data ***
( for summarization tasks, this could be a summary of that chunk; for question-answering tasks, it could be an answer based solely on that chunk). **Then a different prompt is run to combine all the initial outputs.** This is implemented in the LangChain as the MapReduceDocumentsChain.

**Pros:** Can scale to larger documents (and more documents) than StuffDocumentsChain. The calls to the LLM on individual documents are independent and can therefore be parallelized.

**Cons:** Requires many more calls to the LLM than StuffDocumentsChain. Loses some information during the final combining call.

In [481]:
from langchain.chains.summarize import load_summarize_chain
import textwrap

In [482]:
chain = load_summarize_chain(llm,
                             chain_type="map_reduce")


output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary, width=100)
print(wrapped_text)

Daniel Dias reached out to Sandra Regina Barsch from the Sales & CRM Support team about an issue
where an order was initially disapproved but later partially billed, only to revert to disapproved
due to a setting in the CM0101 configuration. He provided a screenshot for establishment 600,
seeking clarification on why the order, blocked on credit on March 8th, proceeded with shipping and
billing, and mentioned generating the spp of the order for analysis.


In [483]:
# for summarizing each part
chain.llm_chain.prompt.template

'Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:'

In [484]:
# for combining the parts
chain.combine_document_chain.llm_chain.prompt.template

'Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:'

In [489]:
chain = load_summarize_chain(llm,
                             chain_type="map_reduce",
                             verbose=True
                             )


output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary,
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)

ValidationError: 1 validation error for MapReduceDocumentsChain
question_prompt
  extra fields not permitted (type=value_error.extra)

### Summarizing with the 'stuff' Chain



### Stuffing
Stuffing is the simplest method, whereby you simply stuff all the related data into the prompt as context to pass to the language model. This is implemented in LangChain as the StuffDocumentsChain.

**Pros:** Only makes a single call to the LLM. When generating text, the LLM has access to all the data at once.

**Cons:** Most LLMs have a context length, and for large documents (or many documents) this will not work as it will result in a prompt larger than the context length.

The main downside of this method is that **it only works one smaller pieces of data.**  Once you are working with many pieces of data, this approach is no longer feasible. The next two approaches are designed to help deal with that.



In [ ]:
chain = load_summarize_chain(llm, chain_type="stuff")

In [ ]:
prompt_template = """Write a concise bullet point summary of the following:


{text}

CONSCISE SUMMARY IN BULLET POINTS:"""

BULLET_POINT_PROMPT = PromptTemplate(template=prompt_template,
                        input_variables=["text"])


In [ ]:
chain = load_summarize_chain(llm,
                             chain_type="stuff",
                             prompt=BULLET_POINT_PROMPT)

output_summary = chain.run(docs)

wrapped_text = textwrap.fill(output_summary,
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)

### Ver 3 With 'map_reduce' with our custom prompt

In [ ]:
chain = load_summarize_chain(llm,
                             chain_type="map_reduce",
                             map_prompt=BULLET_POINT_PROMPT,
                             combine_prompt=BULLET_POINT_PROMPT)

# chain.llm_chain.prompt= BULLET_POINT_PROMPT
# chain.combine_document_chain.llm_chain.prompt= BULLET_POINT_PROMPT

output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary,
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)

In [490]:
# with a custom prompt
prompt_template = """Write a concise summary of the following:


{text}


CONSCISE SUMMARY IN BULLET POINTS:"""

PROMPT = PromptTemplate(template=prompt_template,
                        input_variables=["text"])

## with intermediate steps
chain = load_summarize_chain(llm,
                             chain_type="map_reduce",
                             return_intermediate_steps=True,
                             map_prompt=PROMPT,
                             combine_prompt=PROMPT)

output_summary = chain({"input_documents": docs}, return_only_outputs=True)
wrapped_text = textwrap.fill(output_summary['output_text'],
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)

- Daniel's order was initially rejected but later approved for partial billing.
- Post-billing, the
order was again rejected due to the "Reopen credit evaluation" setting in CM0101.
- Sandra requested
a screenshot of the CM0101 screen from establishment 600 for analysis.
- Daniel provided the
requested screenshot and noted that despite a credit block on 08/03, the order proceeded to shipment
and billing.
- Daniel seeks help to understand the credit issue and has prepared the spp of the
order for analysis.


In [ ]:
wrapped_text = textwrap.fill(output_summary['intermediate_steps'][2],
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)

### With the 'refine' CombineDocument Chain

## Refine
This method involves **an initial prompt on the first chunk of data, generating some output. For the remaining documents, that output is passed in, along with the next document**, asking the LLM to refine the output based on the new document.

**Pros:** Can pull in more relevant context, and may be less lossy than MapReduceDocumentsChain.

**Cons:** Requires many more calls to the LLM than StuffDocumentsChain. The calls are also NOT independent, meaning they cannot be paralleled like MapReduceDocumentsChain. There is also some potential dependencies on the ordering of the documents.

In [ ]:
chain = load_summarize_chain(llm, chain_type="refine")

output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary, width=100)
print(wrapped_text)

In [478]:
prompt_template = """Write a concise summary, in chronological order and 
                     in the Brazilian Portuguese language 
                     Please disregard personal names
                     of the following:


{text}


CONCISE SUMMARY:"""
PROMPT = PromptTemplate(template=prompt_template,
                        input_variables=["text"])

refine_template = (
    "Your job is to produce a final summary\n"
    "We have provided an existing summary up to a certain point: {existing_answer}\n"
    "We have the opportunity to refine the existing summary"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original summary"
    "If the context isn't useful, return the original summary."
)

refine_prompt = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)

chain = load_summarize_chain(llm,
                             chain_type="refine",
                             return_intermediate_steps=True,
                             question_prompt=PROMPT,
                             refine_prompt=refine_prompt)


In [479]:
output_summary = chain({"input_documents": docs}, return_only_outputs=True)
wrapped_text = textwrap.fill(output_summary['output_text'],
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)

Inicialmente, um pedido foi reprovado, mas acabou sendo aprovado forçadamente para permitir o
faturamento parcial. Após o faturamento, o pedido foi novamente reprovado devido à configuração do
campo "Reabre avaliação de crédito" no CM0101. Sandra Regina Barsch, do Suporte Vendas & CRM,
menciona a documentação relevante sobre o campo e solicita esclarecimentos. Daniel Dias, por sua
vez, envia um print da tela CM0101 do estabelecimento 600 e relata que, no dia 08/03, apesar do
bloqueio de crédito, foi possível prosseguir com o embarque e faturamento do pedido. Ele solicita
análise para entender o ocorrido com o crédito.


In [ ]:
wrapped_text = textwrap.fill(output_summary['intermediate_steps'][0],
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)